In [24]:
import json
import pandas as pd
from natsort import natsorted

# Load the JSON data
file_path = '/Users/eolika/Desktop/train_results_901/fold2/summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract relevant data
metrics = data['mean']

# Initialize lists to store the calculated values
classes = natsorted(list(metrics.keys()))
dice_per_class = []
iou_per_class = []
sensitivity_per_class_segmentation = []
specificity_per_class_segmentation = []
sensitivity_per_class_detection = []
specificity_per_class_detection = []

# Calculate Sensitivity and Specificity for segmentation and detection per class
for class_id, metric in natsorted(metrics.items()):
    tp = metric['TP']
    fn = metric['FN']
    fp = metric['FP']
    tn = metric['TN']
    
    # Segmentation sensitivity and specificity
    sensitivity_seg = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity_seg = tn / (tn + fp) if (tn + fp) > 0 else 0

    
    dice_per_class.append(metric['Dice'])
    iou_per_class.append(metric['IoU'])
    sensitivity_per_class_segmentation.append(sensitivity_seg)
    specificity_per_class_segmentation.append(specificity_seg)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Class': classes,
    'Dice': dice_per_class,
    'IoU': iou_per_class,
    'Sensitivity (Segmentation)': sensitivity_per_class_segmentation,
    'Specificity (Segmentation)': specificity_per_class_segmentation,
})

results_df.to_csv(file_path.replace('.json', '.csv'), index=False)

In [25]:
results_df

,Class,Dice,IoU,Sensitivity (Segmentation),Specificity (Segmentation)
0,1,0.982303,0.965880,0.983700,0.999604
1,2,0.815889,0.759176,0.906941,0.999981
2,3,0.943616,0.910461,0.967727,0.999914
3,4,0.957970,0.925346,0.967017,0.999934
4,5,0.909997,0.847034,0.931451,0.999927
5,6,0.974362,0.951408,0.977600,0.999933
6,7,0.839443,0.750605,0.881309,0.999992
7,8,0.840366,0.749707,0.887526,0.999994
8,9,0.703541,0.606570,0.735448,0.999938
9,10,0.474849,0.392634,0.763575,0.999886


### Get mean for summary.csv of differents folds

In [27]:
# Read the CSV files into DataFrames
df_0 = pd.read_csv('/Users/eolika/Desktop/train_results_901/fold0/summary.csv')
df_1 = pd.read_csv('/Users/eolika/Desktop/train_results_901/fold1/summary.csv')
df_2 = pd.read_csv('/Users/eolika/Desktop/train_results_901/fold2/summary.csv')

# Concatenate the DataFrames
combined_df = pd.concat([df_0, df_1, df_2])

# Calculate the mean for each class
mean_results_df = combined_df.groupby('Class').mean().reset_index()

results_df.to_csv('/Users/eolika/Desktop/train_results_901/summary.csv', index=False)

mean_results_df

,Class,Dice,IoU,Sensitivity (Segmentation),Specificity (Segmentation)
0,1,0.981569,0.964349,0.983850,0.999511
1,2,0.835603,0.777254,0.920790,0.999980
2,3,0.941001,0.906420,0.965846,0.999921
3,4,0.954193,0.921024,0.965429,0.999931
4,5,0.914366,0.850355,0.929419,0.999928
5,6,0.973764,0.949948,0.976162,0.999908
6,7,0.831517,0.740847,0.875915,0.999992
7,8,0.838030,0.745077,0.874245,0.999994
8,9,0.666864,0.574112,0.728298,0.999920
9,10,0.485637,0.400474,0.744688,0.999892


### Get mean for results.json of differents folds

In [47]:
# Read the CSV files into DataFrames
df_0 = pd.read_json('/Users/eolika/Desktop/train_results_901/fold0/results.json').drop(columns=['subjects']).drop_duplicates().T
df_1 = pd.read_json('/Users/eolika/Desktop/train_results_901/fold1/results.json').drop(columns=['subjects']).drop_duplicates().T
df_2 = pd.read_json('/Users/eolika/Desktop/train_results_901/fold2/results.json').drop(columns=['subjects']).drop_duplicates().T

# Combine the DataFrames
combined_df = pd.concat([df_0, df_1, df_2], axis=1)

# Calculate the mean across the folds
mean_results_df = combined_df.mean(axis=1).reset_index()
mean_results_df.columns = ['Metric', 'Mean Value']

# Split the Metric column into anatomical structure and metric type
mean_results_df[['Class', 'Metric Type']] = mean_results_df['Metric'].str.rsplit(' ', n=1, expand=True)

# Pivot the DataFrame to have separate columns for Dice and SurfaceDice3
mean_results_df_pivot = mean_results_df.pivot(index='Class', columns='Metric Type', values='Mean Value').reset_index()
mean_results_df_pivot.columns.name = None  # Remove the axis name

# Rename the columns as required
mean_results_df_pivot = mean_results_df_pivot.rename(columns={'dice': 'Dice', 'surface_dice_3': 'SurfaceDice3'})

mean_results_df_pivot.to_csv('/Users/eolika/Desktop/train_results_901/results.csv', index=False)

mean_results_df_pivot

,Class,Dice,SurfaceDice3
0,adrenal_gland_formation,0.783667,0.788000
1,adrenal_gland_left,0.853333,0.971000
2,adrenal_gland_right,0.858667,0.979667
3,gallbladder,0.863000,0.918667
4,kidney_formation,0.739333,0.798000
5,kidney_left,0.945667,0.968667
6,kidney_right,0.958667,0.980000
7,liver,0.981333,0.980667
8,liver_formation,0.663000,0.765667
9,pancreas,0.917333,0.964333
